# Projeto #1 - Mineração de padrões frequentes

<p style='text-align: justify;'><font size=3.>Análise de acidentes nas rodovias federais brasileira entre 2007 e 2021.</font></p>

> <p style='text-align: justify;'><font size=2.85>Esse projeto utiliza a versão 0.17.0 do mlxtend.</font></p>

In [2]:
import mlxtend
print(mlxtend.__version__)

0.17.0


### Bibliotecas básicas e outros imports

In [3]:
import warnings
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 200)
np.set_printoptions(suppress=True, precision=3)

%run modules/text.py

### Carregamento dos dados

In [4]:
df = pd.read_csv('dataset/cleaned_acidentes_rodovias_brasileiras_2007_a_2021.csv', low_memory=False, encoding='utf-8')
print(f'Linhas: {df.shape[0]} | Coluna:  {df.shape[1]}')
df.head(1)

Linhas: 1894736 | Coluna:  24


,id,dia_semana,uf,br,municipio,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,sentido_via,condicao_metereologica,tipo_pista,tracado_via,uso_solo,tipo_veiculo,tipo_envolvido,estado_fisico,data,mes,ano,estacao_do_ano,regiao_pais,estacao,clima
0,83123214.0,quarta,SC,280,Rio negrinho,outras,saida de pista,ignorada,plena noite,decrescente,chuva,simples,curva,urbano,automovel,condutor,ignorada,2014-01-01,1,2014,verão,sul,3,1


### Remoção valores inválidos

In [5]:
df.drop(df[df['tipo_veiculo'] == 'nao identificado'].index, inplace=True)
df.drop(df[df['tipo_veiculo'] == 'nao informado'].index, inplace=True)
df.drop(df[df['classificacao_acidente'] == 'ignorada'].index, inplace=True)
df.drop(df[df['condicao_metereologica'] == 'ignorada'].index, inplace=True)
print(f'Linhas: {df.shape[0]} | Coluna:  {df.shape[1]}')

Linhas: 1876750 | Coluna:  24


### Geração de features

#### Agrupamento tipos de acidentes

In [6]:
df['tipo_acidente'] = df['tipo_acidente'].apply(mapTiposDeAcidentes)

#### Agrupamento fase do dia

In [7]:
df['fase_dia'] = df['fase_dia'].map(mapFase)

#### Agrupamento dias da semana

In [8]:
df['dia_semana'] = df['dia_semana'].apply(mapDiasDaSemana)

#### Agrupamento condição meteorológica

In [9]:
df['condicao_clima'] = df['condicao_metereologica'].map(mapClima)

### Agrupamento tamanho veículos

In [10]:
df['porte_veiculo'] = df['tipo_veiculo'].apply(mapTamanhoVeiculos)

### Seleção dos recursos para análise

In [11]:
cols = [
        'classificacao_acidente',
        'tipo_acidente', 
        'fase_dia',	
        'condicao_clima', 
        'tracado_via',
        'porte_veiculo', 
        'estacao_do_ano',
        'dia_semana'
        ]
data = df[cols]
data.head()

,classificacao_acidente,tipo_acidente,fase_dia,condicao_clima,tracado_via,porte_veiculo,estacao_do_ano,dia_semana
1,sem vitimas,colisão,noite,tempo ruim,reta,pequeno porte,verão,dia útil
2,sem vitimas,atropelamento,noite,ensolarado,reta,pequeno porte,verão,dia útil
3,sem vitimas,colisão,noite,ensolarado,cruzamento,pequeno porte,verão,dia útil
4,com vitimas feridas,colisão,noite,nublado,reta,pequeno porte,verão,dia útil
5,sem vitimas,colisão,noite,nublado,reta,pequeno porte,verão,dia útil


### Agrupamento

#### 1º cluster: Sem vítimas

In [12]:
sem_vitimas = data.query('classificacao_acidente=="sem vitimas"')
sem_vitimas.drop(['classificacao_acidente'], axis=1, inplace=True)
print(f'Linhas: {sem_vitimas.shape[0]} | Coluna:  {sem_vitimas.shape[1]}')

Linhas: 968151 | Coluna:  7


#### 2º cluster: Vítimas feridas



In [13]:
vitimas_feridas = data.query('classificacao_acidente=="com vitimas feridas"')
vitimas_feridas.drop(['classificacao_acidente'], axis=1, inplace=True)
print(f'Linhas: {vitimas_feridas.shape[0]} | Coluna:  {vitimas_feridas.shape[1]}')

Linhas: 797703 | Coluna:  7


#### 3º cluster: Vítimas fatais

In [14]:
vitimas_fatais = data.query('classificacao_acidente=="com vitimas fatais"')
vitimas_fatais.drop(['classificacao_acidente'], axis=1, inplace=True)
print(f'Linhas: {vitimas_fatais.shape[0]} | Coluna:  {vitimas_fatais.shape[1]}')

Linhas: 78625 | Coluna:  7


## Mineração das regras

#### Enconder da base

In [15]:
def transactionEncoderData(dados:np.ndarray) -> pd.DataFrame:
    transEncoder = TransactionEncoder()
    a_data = transEncoder.fit(dados).transform(dados)
    dados = pd.DataFrame(a_data, columns=transEncoder.columns_)
    return dados*1

#### Itens frequentes
> <p style='text-align: justify;'><font size=2.85>Fpgrowth foi escolhido por ser mais eficiente</font></p>

In [16]:
def getFrequentItemsets(encoderData:pd.DataFrame, minsup:float=0.3, useColnames:bool=True) -> pd.DataFrame:
    return fpgrowth(encoderData, min_support=minsup, use_colnames=useColnames)

#### Regras

In [17]:
def getAssociationRules(itemsets:pd.DataFrame, metric:str='lift', minThreshold:int=1) -> pd.DataFrame:
    return association_rules(itemsets, metric=metric, min_threshold=minThreshold)

### Acidentes sem vítimas

In [18]:
sem_vitimas = transactionEncoderData(sem_vitimas.values)
frequent_itemsets = getFrequentItemsets(sem_vitimas)
rules_sem_vitimas = getAssociationRules(frequent_itemsets)
rules_sem_vitimas = rules_sem_vitimas.sort_values(by='lift', ascending=False)
rules_sem_vitimas[['antecedents', 'consequents', 'confidence', 'lift', 'leverage', 'conviction']].query('confidence>=0.70').reset_index(drop=True)

,antecedents,consequents,confidence,lift,leverage,conviction
0,"(ensolarado, dia)","(dia útil, colisão)",0.720041,1.151588,0.040476,1.338555
1,"(dia, reta)","(dia útil, colisão)",0.718397,1.148959,0.043670,1.330741
2,"(dia, dia útil, reta)",(colisão),0.805858,1.132427,0.039390,1.485406
3,"(ensolarado, dia útil, dia)",(colisão),0.805672,1.132165,0.035895,1.483984
4,"(ensolarado, dia útil, colisão)",(dia),0.751962,1.130198,0.035423,1.349244
5,"(dia, reta)",(colisão),0.798857,1.122588,0.040903,1.433702
6,"(ensolarado, dia)",(colisão),0.797706,1.120971,0.036762,1.425544
7,"(ensolarado, dia útil, reta)",(colisão),0.792497,1.113652,0.032776,1.389764
8,"(ensolarado, pequeno porte)",(colisão),0.791467,1.112204,0.031334,1.382896
9,"(ensolarado, colisão)",(dia),0.737754,1.108844,0.033439,1.276146


> <p style='text-align: justify;'><font size=2.85>Nos resultados da análise das regras de associação para acidentes sem vítimas, a regra com o maior valor de Lift é 1.15 e mostra que há, principalmente, colisões que ocorrem em dias ensolarados durante a semana. Avaliando as demais regras, temos que a maioria dos acidentes ocorrem durante o dia e envolvem veículos de pequeno porte sendo o tipo de acidente mais comum as colisões em pistas retas.</font></p>

### Acidentes com vítimas feridas

In [19]:
vitimas_feridas = transactionEncoderData(vitimas_feridas.values)
frequent_itemsets = getFrequentItemsets(vitimas_feridas)
rules_vitimas_feridas = getAssociationRules(frequent_itemsets)
rules_vitimas_feridas = rules_vitimas_feridas.sort_values(by='lift', ascending=False)
rules_vitimas_feridas[['antecedents', 'consequents', 'confidence', 'lift', 'leverage', 'conviction']].query('confidence>=0.60').reset_index(drop=True)

,antecedents,consequents,confidence,lift,leverage,conviction
0,"(ensolarado, dia útil)",(dia),0.660830,1.074899,0.025181,1.135762
1,(colisão),"(dia útil, reta)",0.609595,1.074810,0.024682,1.108682
2,"(dia útil, reta)",(colisão),0.625230,1.074810,0.024682,1.116119
3,"(ensolarado, dia útil)",(colisão),0.617033,1.060720,0.019316,1.092231
4,"(dia útil, colisão)",(reta),0.720159,1.059761,0.019997,1.145120
5,(colisão),(reta),0.719559,1.058877,0.023274,1.142668
6,(reta),(colisão),0.615961,1.058877,0.023274,1.089183
7,"(dia útil, reta)",(ensolarado),0.689973,1.053301,0.019803,1.112620
8,"(ensolarado, dia útil)",(reta),0.715585,1.053030,0.019707,1.126704
9,"(dia útil, colisão)",(dia),0.646206,1.051111,0.015472,1.088815


> <p style='text-align: justify;'><font size=2.85>Para o cluster de vítimas feridas, temos a principal regra com um lift de 1.06 indicando que quando ocorrem colisões em pistas retas, normalmente durante a semana, é provável que os envolvidos sofram algum tipo de ferimento. Também temos nesse grupo que a maioria dos acidentes que causam ferimentos ocorrem em condições de dias ensolarados ou durante o período do dia. Acidentes desse cluster envolvem veículos de pequeno porte.</font></p>

### Acidentes com vítimas fatais

In [21]:
vitimas_fatais = transactionEncoderData(vitimas_fatais.values)
frequent_itemsets = getFrequentItemsets(vitimas_fatais)
rules_vitimas_fatais = getAssociationRules(frequent_itemsets)
rules_vitimas_fatais = rules_vitimas_fatais.sort_values(by='lift', ascending=False)
rules_vitimas_fatais[['antecedents', 'consequents', 'confidence', 'lift', 'leverage', 'conviction']].query('confidence>=0.60').reset_index(drop=True)

,antecedents,consequents,confidence,lift,leverage,conviction
0,(noite),(reta),0.735494,1.054720,0.019965,1.144262
1,(dia),(colisão),0.641352,1.052148,0.015156,1.088631
2,(dia),(ensolarado),0.696116,1.044407,0.014112,1.097398
3,"(ensolarado, dia útil)",(reta),0.725558,1.040472,0.014785,1.102835
4,(reta),(ensolarado),0.692548,1.039053,0.018151,1.084661
5,(ensolarado),(reta),0.724568,1.039053,0.018151,1.098873
6,"(dia útil, reta)",(ensolarado),0.690711,1.036297,0.013313,1.078221
7,(dia),(dia útil),0.815701,1.031698,0.011949,1.135985
8,(pequeno porte),(reta),0.710476,1.018843,0.005890,1.045385
9,(colisão),(reta),0.706637,1.013339,0.005670,1.031707


> <p style='text-align: justify;'><font size=2.85>Em acidentes com vítimas fatais, a nossa principal regra, com lift de 1.05, nos mostra que nesse cluster os acidentes ocorrem, principalmente, durante a noite e em pistas com traçado reto. Boa parte também ocorrem durante a semana, no período do dia e em dias ensolarados. O tipo de acidente mais recorrente são as colisões e essas ocorrências também envolvem veículos de pequeno porte.</font></p>